In [1]:
import docx
import re
from langchain_core.documents import Document

### Обработка файла 1
---

In [9]:
def extract_situations_from_docx(file_path):
    document = docx.Document(file_path)
    situations = {}

    current_number = 0
    cur_text = ""
    situations[current_number] = {}

    for para in document.paragraphs:
        numbering = para._p.pPr.numPr if para._p.pPr is not None else None
        para_text = para.text.strip()

        if numbering is not None:
            
            situations[current_number]["assessment"] = cur_text
            current_number += 1
            situations[current_number] = {}
            cur_text = ""

            if para_text.startswith("Ситуация:"):
                cur_text += para_text.replace("Ситуация:", "").strip()
        else:
            if para_text.startswith("Оценка критичности, рекомендации:"):
                situations[current_number]["situation"] = cur_text
                cur_text = ""
                cur_text += para_text.replace("Оценка критичности, рекомендации:", "").strip()
            else:
                cur_text += para_text
    situations[current_number]["assessment"] = cur_text
    situations.pop(0, None)

    return situations

In [10]:
file_path = '../data/1_file.docx'
analysis_situations = extract_situations_from_docx(file_path)

In [13]:
# analysis_situations

In [14]:
analysis_docs = []
for key in analysis_situations:
    analysis_docs.append(
        Document(
            page_content=analysis_situations[key]["situation"], 
            metadata={
                "situation": analysis_situations[key]["situation"], 
                "assessment": analysis_situations[key]["assessment"]
            },
            id=2*key-1
        )
    )
    analysis_docs.append(
        Document(
            page_content=analysis_situations[key]["assessment"], 
            metadata={
                "situation": analysis_situations[key]["situation"], 
                "assessment": analysis_situations[key]["assessment"]
            },
            id=2*key
        )
    )

In [16]:
# analysis_docs

### Обработка файла 2
---

In [21]:
def extract_situations_from_docx_2(file_path):
    document = docx.Document(file_path)
    situations = {}

    current_number = 0
    cur_text = ""
    situations[current_number] = {}

    for para in document.paragraphs:
        numbering = para._p.pPr.numPr if para._p.pPr is not None else None
        para_text = para.text.strip()

        if numbering is not None:
            
            situations[current_number]["action"] = cur_text
            current_number += 1
            situations[current_number] = {}
            cur_text = ""

            if para_text.startswith("Ситуация:"):
                cur_text += para_text.replace("Ситуация:", "").strip()
        else:
            if para_text.startswith("Необходимые мероприятия юридического подразделения:"):
                situations[current_number]["situation"] = cur_text
                cur_text = ""
                cur_text += para_text.replace("Необходимые мероприятия юридического подразделения:", "").strip()
            else:
                cur_text += para_text
    situations[current_number]["action"] = cur_text
    situations.pop(0, None)

    return situations

In [22]:
file_path = '../data/2_file.docx'
procedures_situations = extract_situations_from_docx_2(file_path)

In [24]:
procedures_docs = []
for key in procedures_situations:
    procedures_docs.append(
        Document(
            page_content=procedures_situations[key]["situation"], 
            metadata={
                "situation": procedures_situations[key]["situation"], 
                "action": procedures_situations[key]["action"]
            },
            id=2*key-1
        )
    )
    procedures_docs.append(
        Document(
            page_content=procedures_situations[key]["action"], 
            metadata={
                "situation": procedures_situations[key]["situation"], 
                "action": procedures_situations[key]["action"]
            },
            id=2*key
        )
    )

### Credentials

In [26]:
import os
import httpx
from dotenv import load_dotenv

load_dotenv()

proxy_url = os.environ.get("OPENAI_PROXY_URL")

### Initialization

In [27]:
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model="text-embedding-3-large", http_client=httpx.Client(proxy=proxy_url))

### Init chroma db
---

In [28]:
import chromadb
from langchain_chroma import Chroma

persistent_client  = chromadb.PersistentClient(path="chromadb_db/")

In [29]:
vector_store_analysis = Chroma(client=persistent_client, collection_name="judicial_acts_analysis", embedding_function=embedding_function)
vector_store_procedures = Chroma(client=persistent_client, collection_name="rejection_procedures", embedding_function=embedding_function)

In [31]:
# %%time

# vector_store_analysis.add_documents(documents=analysis_docs)
# vector_store_procedures.add_documents(documents=procedures_docs)

In [32]:
vector_store_analysis.similarity_search("Долг", k=3)

[Document(id='42', metadata={'assessment': 'Некритично, если расхождения сумм задолженности в диапазоне 250 рублей (включительно). В остальных случаях критично.', 'situation': 'Несовпадение сумм задолженности как в разбивке, так и в общем итоге в резолютивной части судебного акта с РЦИ (расчет, расчет цены иска).'}, page_content='Некритично, если расхождения сумм задолженности в диапазоне 250 рублей (включительно). В остальных случаях критично.'),
 Document(id='1', metadata={'assessment': 'Некритично. Необходимо произвести расчет госпошлины для каждого обязательства и указать сумму в карточке дела и в запросе в АС ВРМ «Правовое обеспечение».', 'situation': 'Судебный акт вынесен по нескольким договорам, сумма госпошлины (г/п, гп) указана общей суммой без разбивки.'}, page_content='Судебный акт вынесен по нескольким договорам, сумма госпошлины (г/п, гп) указана общей суммой без разбивки.'),
 Document(id='39', metadata={'assessment': 'Некритично, если в итоговой сумме задолженности размер

In [33]:
vector_store_procedures.similarity_search("отказ", k=3)

[Document(id='3', metadata={'action': 'проверить, что судебный акт вынесен по категории дел «Двойное просуживание». Проверить наличие спец.метки в кейсе, при отсутствии нужно проставить. Выставить запрос в ПЦП ЦУЗ для урегулирования суммовой разницы с комментарием «#Двойное просуживание#суммовая разница/судом частично удовлетворены исковые требования, Требуется урегулировать: ОД -________ ₽, % - _________». В случае, если в исковом заявлении не заявлены неустойки, а в РЦИ неустойки присутствуют, в комментарий для ПЦП ЦУЗ необходимо включить информацию об урегулировании незаявленных неустоек.\nПосле выставления задачи в ПЦП ЦУЗ для урегулирования суммовой разницы необходимо передать документы в ПСИП.В случаях возникновения дополнительных вопросов необходимо обратиться к методологам аппарата ТБ.', 'situation': 'получен отказ от ПЦП ОЦ по причине: «Суммовая разница»'}, page_content='получен отказ от ПЦП ОЦ по причине: «Суммовая разница»'),
 Document(id='1', metadata={'action': 'необходимо